In [ ]:
import json
import math
from typing import List, Dict, Any

class DroneDeliveryOptimizer:
    def __init__(self, input_data: Dict[str, Any]):
        self.city_grid_size = input_data['city']['grid_size']
        self.no_fly_zone = input_data
        self.drones = input_data['drones']['fleet']
        self.orders = input_data['orders']

    def calculate_distance(self, start: tuple, end: tuple) -> float:
        return abs(start[0] - end[0]) + abs(start[1] - end[1])

    def is_drone_suitable(self, drone: Dict[str, Any], order: Dict[str, Any], 
                        current_path: List[tuple], current_weight: float) -> bool:
        # Basic availability check
        if not drone['available']:
            return False
            
        # Check weight capacity
        new_total_weight = current_weight + order['package_weight']
        if new_total_weight > drone['max_payload']:
            return False
            
        # Check distance constraints with the new order included
        order_location = (order['delivery_x'], order['delivery_y'])
        
        # Create test path with new order and return to depot
        test_path = current_path + [order_location, (0, 0)]
        
        # Calculate total distance of the test path
        path_distance = 0
        for i in range(1, len(test_path)):
            path_distance += self.calculate_distance(test_path[i-1], test_path[i])
            
        # Check if the path exceeds drone's maximum range
        if path_distance > drone['max_distance']:
            return False
            
        return True
    
    def optimize_deliveries(self) -> Dict[str, List[Dict[str, Any]]]:
        assignments = []
        used_orders = set()

        sorted_drones = sorted(
            [drone for drone in self.drones if drone['available']],
            key=lambda d: (d['max_payload'], d['speed']),
            reverse=True
        )

        for drone in sorted_drones:
            drone_orders = []
            drone_total_weight = 0
            drone_locations = [(0, 0)]  # Start at depot
            
            remaining_orders = [
                order for order in self.orders
                if order['id'] not in used_orders
            ]
            remaining_orders.sort(key=lambda o: (o['deadline'], -o['package_weight']))

            for order in remaining_orders:
                # Use the improved suitability check that considers the current path
                if not self.is_drone_suitable(drone, order, drone_locations, drone_total_weight):
                    continue
                    
                # Order is acceptable, add it
                drone_orders.append(order['id'])
                used_orders.add(order['id'])
                drone_total_weight += order['package_weight']
                order_location = (order['delivery_x'], order['delivery_y'])
                drone_locations.append(order_location)  # Update drone's path
                    
            # Calculate final path distance (including return to depot if not already there)
            if drone_orders:
                if drone_locations[-1] != (0, 0):
                    drone_locations.append((0, 0))  # Return to depot
                    
                drone_total_distance = 0
                for i in range(1, len(drone_locations)):
                    drone_total_distance += self.calculate_distance(drone_locations[i-1], drone_locations[i])
                    
                assignments.append({
                    "drone": drone['id'],
                    "orders": drone_orders,
                    "total_distance": round(drone_total_distance)
                })

        return {"assignments": assignments}

def main(input_file: str, output_file: str):
    with open(input_file, 'r') as f:
        input_data = json.load(f)
    optimizer = DroneDeliveryOptimizer(input_data)
    result = optimizer.optimize_deliveries()

    with open(output_file, 'w') as f:
        json.dump(result, f, indent=2)

    print("\nFinal Result:", json.dumps(result, indent=2))

if __name__ == "__main__":
    main('D:/GUVI/Tarka labs/inputs/case4/input.json', 'output.json')

2.5
4.5
6.0
1.5

Final Result: {
  "assignments": [
    {
      "drone": "drone1",
      "orders": [
        "order3",
        "order1"
      ],
      "total_distance": 18
    },
    {
      "drone": "drone2",
      "orders": [
        "order2"
      ],
      "total_distance": 8
    }
  ]
}


In [2]:
import json
import math
from typing import List, Dict, Any, Tuple

class DroneDeliveryOptimizer:
    def __init__(self, input_data: Dict[str, Any]):
        self.city_grid_size = input_data['city']['grid_size']
        self.drones = input_data['drones']['fleet']
        self.orders = input_data['orders']
        
        # Handle no-fly zone if present in input data
        self.no_fly_zones = []
        if 'no_fly_zone' in input_data:
            zone = input_data['no_fly_zone']
            # Extract coordinates from no-fly zone definition
            x_min = min(zone['point_1'][0], zone['point_2'][0], zone['point_3'][0], zone['point_4'][0])
            x_max = max(zone['point_1'][0], zone['point_2'][0], zone['point_3'][0], zone['point_4'][0])
            y_min = min(zone['point_1'][1], zone['point_2'][1], zone['point_3'][1], zone['point_4'][1])
            y_max = max(zone['point_1'][1], zone['point_2'][1], zone['point_3'][1], zone['point_4'][1])
            self.no_fly_zones.append((x_min, y_min, x_max, y_max))

    def calculate_distance(self, start: tuple, end: tuple) -> float:
        """Calculate Manhattan distance between two points"""
        return abs(start[0] - end[0]) + abs(start[1] - end[1])
    
    def intersects_no_fly_zone(self, point1: Tuple[int, int], point2: Tuple[int, int]) -> bool:
        """
        Check if a path segment between point1 and point2 intersects with any no-fly zone.
        Using a simplified approach based on Manhattan distance path segments.
        """
        # If no no-fly zones, return False immediately
        if not self.no_fly_zones:
            return False
            
        # For Manhattan distance paths, we need to check if any point in the path
        # lies within a no-fly zone
        x1, y1 = point1
        x2, y2 = point2
        
        # Get the points along the Manhattan path
        path_points = []
        
        # First move horizontally
        x_direction = 1 if x2 > x1 else -1 if x2 < x1 else 0
        for x in range(x1, x2 + x_direction, x_direction) if x_direction != 0 else [x1]:
            path_points.append((x, y1))
            
        # Then move vertically (excluding the starting point which is already included)
        y_direction = 1 if y2 > y1 else -1 if y2 < y1 else 0
        for y in range(y1 + y_direction, y2 + y_direction, y_direction) if y_direction != 0 else []:
            path_points.append((x2, y))
        
        # Check if any point in the path is inside a no-fly zone
        for point in path_points:
            for zone in self.no_fly_zones:
                x_min, y_min, x_max, y_max = zone
                if x_min <= point[0] <= x_max and y_min <= point[1] <= y_max:
                    return True
                    
        return False

    def calculate_path_distance(self, path: List[Tuple[int, int]]) -> float:
        """Calculate total distance along a path with no-fly zone considerations"""
        if len(path) <= 1:
            return 0
            
        total_distance = 0
        for i in range(1, len(path)):
            # Check if direct path intersects no-fly zone
            if self.intersects_no_fly_zone(path[i-1], path[i]):
                # If the path intersects, find a detour
                detour_points = self.find_detour(path[i-1], path[i])
                
                # Calculate distance along the detour
                for j in range(1, len(detour_points)):
                    total_distance += self.calculate_distance(detour_points[j-1], detour_points[j])
            else:
                # If no intersection, use direct path distance
                total_distance += self.calculate_distance(path[i-1], path[i])
                
        return total_distance
        
    def find_detour(self, start: Tuple[int, int], end: Tuple[int, int]) -> List[Tuple[int, int]]:
        """Find a path that avoids no-fly zones"""
        # This is a simplified implementation that finds detours
        # by going around the no-fly zones
        detour = [start]
        
        for zone in self.no_fly_zones:
            x_min, y_min, x_max, y_max = zone
            
            # Check if the direct path intersects this zone
            if not self.intersects_no_fly_zone(start, end):
                # If it doesn't intersect, no need for a detour
                continue
                
            # Determine detour points - we'll go around the zone
            # Option 1: Go around horizontally (left/right side)
            if start[1] < y_min and end[1] < y_min:  # Both points below zone
                detour.append((start[0], start[1]))
                detour.append((end[0], end[1]))
            elif start[1] > y_max and end[1] > y_max:  # Both points above zone
                detour.append((start[0], start[1]))
                detour.append((end[0], end[1]))
            else:
                # Option 2: Go around vertically (top/bottom)
                if start[0] < x_min:  # Start is to the left of zone
                    # Go around the left side
                    detour.append((x_min - 1, start[1]))
                    detour.append((x_min - 1, end[1]))
                else:  # Start is to the right of zone
                    # Go around the right side
                    detour.append((x_max + 1, start[1]))
                    detour.append((x_max + 1, end[1]))
                    
        detour.append(end)
        return detour

    def is_drone_suitable(self, drone: Dict[str, Any], order: Dict[str, Any], 
                         current_path: List[tuple], current_weight: float) -> bool:
        """
        Check if a drone can handle an order, considering existing assigned orders.
        
        Args:
            drone: The drone data
            order: The order to evaluate
            current_path: Current planned path including all previous assigned deliveries
            current_weight: Total weight of previously assigned packages
            
        Returns:
            bool: Whether this drone can handle this order given current assignments
        """
        # Basic availability check
        if not drone['available']:
            return False
            
        # Check weight capacity
        new_total_weight = current_weight + order['package_weight']
        if new_total_weight > drone['max_payload']:
            return False
            
        # Check distance constraints with the new order included
        order_location = (order['delivery_x'], order['delivery_y'])
        
        # Create test path with new order and return to depot
        test_path = current_path + [order_location, (0, 0)]
        
        # Calculate total distance of the test path, considering no-fly zones
        path_distance = self.calculate_path_distance(test_path)
            
        # Check if the path exceeds drone's maximum range
        if path_distance > drone['max_distance']:
            return False
            
        return True

    def optimize_deliveries(self) -> Dict[str, List[Dict[str, Any]]]:
        assignments = []
        used_orders = set()

        sorted_drones = sorted(
            [drone for drone in self.drones if drone['available']],
            key=lambda d: (d['max_payload'], d['speed']),
            reverse=True
        )

        for drone in sorted_drones:
            drone_orders = []
            drone_total_weight = 0
            drone_locations = [(0, 0)]  # Start at depot
            
            # Changed order prioritization to distance first, then deadline
            remaining_orders = [
                order for order in self.orders
                if order['id'] not in used_orders
            ]
            
            # Calculate distance for each order from depot
            for order in remaining_orders:
                order_location = (order['delivery_x'], order['delivery_y'])
                order['distance_from_depot'] = self.calculate_distance((0, 0), order_location)
                
            # Sort by distance first, then by deadline
            remaining_orders.sort(key=lambda o: (o['distance_from_depot'], o['deadline']))

            for order in remaining_orders:
                # Use the improved suitability check that considers the current path
                if not self.is_drone_suitable(drone, order, drone_locations, drone_total_weight):
                    continue
                    
                # Order is acceptable, add it
                drone_orders.append(order['id'])
                used_orders.add(order['id'])
                drone_total_weight += order['package_weight']
                order_location = (order['delivery_x'], order['delivery_y'])
                drone_locations.append(order_location)  # Update drone's path
                    
            # Calculate final path distance (including return to depot if not already there)
            if drone_orders:
                if drone_locations[-1] != (0, 0):
                    drone_locations.append((0, 0))  # Return to depot
                    
                drone_total_distance = self.calculate_path_distance(drone_locations)
                
                # Calculate delivery time in minutes (assuming distance units are in minutes)
                drone_speed = drone['speed']
                delivery_time = drone_total_distance / drone_speed
                    
                assignments.append({
                    "drone": drone['id'],
                    "orders": drone_orders,
                    "total_distance": round(drone_total_distance),
                    "delivery_time_minutes": round(delivery_time)
                })

        return {"assignments": assignments}

def main(input_file: str, output_file: str):
    with open(input_file, 'r') as f:
        input_data = json.load(f)
    optimizer = DroneDeliveryOptimizer(input_data)
    result = optimizer.optimize_deliveries()

    with open(output_file, 'w') as f:
        json.dump(result, f, indent=2)

    print("\nFinal Result:", json.dumps(result, indent=2))

if __name__ == "__main__":
    main('D:/GUVI/Tarka labs/inputs/case4/input.json', 'output.json')


Final Result: {
  "assignments": [
    {
      "drone": "drone1",
      "orders": [
        "order2",
        "order3"
      ],
      "total_distance": 14,
      "delivery_time_minutes": 14
    },
    {
      "drone": "drone2",
      "orders": [
        "order1"
      ],
      "total_distance": 14,
      "delivery_time_minutes": 9
    }
  ]
}
